**本项目基于kaggle网站上「布伦特原油价格」数据集预测未来价格走向**  
原网站数据链接为[Brent Oil Prices | Kaggle](https://www.kaggle.com/mabusalah/brent-oil-prices/kernels)

In [1]:

# %%
from IPython.core.interactiveshell import InteractiveShell
import seaborn as sns
import os
import numpy as np
import pandas as pd
import scipy as sp
from datetime import datetime
import time
import random
import matplotlib.pyplot as plt
from sklearn import preprocessing
import pandas as pd
# import pandas_profile
import numpy as np
import joblib
from dateutil.relativedelta import relativedelta
import warnings

warnings.filterwarnings('ignore')

InteractiveShell.ast_node_interactivity = "all"

from tqdm import tnrange, tqdm_notebook

In [15]:
from time import time

In [2]:
# 解压数据集
import zipfile

In [8]:
with zipfile.ZipFile('Brent Crude.zip', 'r') as zip_f:
    zip_f.extractall('./')

In [9]:
!ls

Brent Crude.zip    BrentOilPrices.csv LSTM_seq.ipynb


# 了解数据总体情况

In [11]:
# 加载文件
df = pd.read_csv('BrentOilPrices.csv')
df.head(3)
df.shape
df.info()

Date  Price
0  20-May-87  18.63
1  21-May-87  18.45
2  22-May-87  18.55

(8360, 2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8360 entries, 0 to 8359
Data columns (total 2 columns):
Date     8360 non-null object
Price    8360 non-null float64
dtypes: float64(1), object(1)
memory usage: 130.7+ KB


数据的总体情况  
- shape为(8360, 2)
- 两列分别为`Date`和`Price`

In [ ]:
tmp.plot

In [23]:
tmp=df.head(100)
plt.figure(figsize=(18, 6))
plt.style.use('ggplot')
plt.plot_date(tmp.Date, tmp.Price, xdate=True, linestyle='solid', marker=None)

<Figure size 1296x432 with 0 Axes>

可以看到价格随着日期而发生波动

# 背景了解

## LSTM原理  
lstm全称为**long short term memory network**，是反向传播并且克服了梯度消失问题的循环神经网络(recurrent network)，不同于传统的循环神经网络是基于神经元联接，`LSTM`层与层之间是通过`memory block`进行连接。  
一个`block`里面有多个gate来控制开关，包括：
- Forget Gate: conditionally decides what information to throw away from the block.  
- Input Gate: conditionally decides which values from the input to update the memory state.  
- Output Gate: conditionally decides what to output based on input and the memory of the block.  

本质上是根据输入信息，block结合当前的state可以智能地判断是否用input的哪些信息进行更新，然后对应更新和保留当前的状态

每个`gate`都是一个一个`sigmoid activation`，新的`input`是一个sequence，input流入到block中，block根据输入，决定添加哪些信息并更新状态

## 问题解析
这个问题本质上可以转化为工程问题，也就是生成两列，一列是上个月的数据，一列是本月，根据上个月的数据预测下个月的数据

# 数据处理部分

- 导入数据
- 数据缩放
- 划分训练集和测试集
- 构建`lstm`指定的数据结构

In [81]:
# 先导入数据
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.


In [25]:
# 设置随机种子
np.random.seed(2020)

In [26]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [32]:
?MinMaxScaler.fit

In [34]:
df['Price'].values.shape

(8360,)

In [36]:
df['Price'].values.reshape(-1,1).shape

(8360, 1)

In [37]:
# 数据缩放, LSTM里面的激活函数`sigmoid`和`tanh`函数对数据的量级很敏感，因此在进入模型之前需要先进行归一化
price_value = df['Price'].values.reshape(-1,1)

scaler = MinMaxScaler()
scaler.fit(price_value)

scale_price = scaler.transform(price_value)


MinMaxScaler(copy=True, feature_range=(0, 1))

In [39]:
max(scale_price)
min(scale_price)

array([1.])

array([0.])

In [40]:
# 划分数据集，按照时序划分
train_size = int(len(scale_price)*0.67)
test_size = len(scale_price) - train_size

train_data = scale_price[:train_size]
test_data = scale_price[train_size:]

In [41]:
train_data.shape
test_data.shape

(5601, 1)

(2759, 1)

In [49]:
train_data[1:4,0]

array([0.0693363 , 0.07007786, 0.07044865])

`lstm`接受的数据结构是`(n_samples, n_time_step, n_features)`这样的数据结构，第一个是样本数量，相当于是记录行数；第二个是时间步，也就是用多大的时间窗口来预测未来，第三个就是每个时间节点可以观测到的变量维度

In [73]:
def get_look_back_data(dataset, lookback = 1):
    # 指定窗口的大小lookback，返回对应的X和Y
    X = []
    Y = []
    for i in range(len(dataset) - lookback -1): #在最后只能生成这么多个数据，可以想象下时间窗口滑到最后，最终预测值个数为1，加上窗口大小为lookback，因此index i只用到达len(dataset) - lookback -1即可
        a = dataset[i:(i+lookback), 0] 
        b = dataset[i+lookback, 0]
        
        X.append(a)
        Y.append(b)
    return np.array(X), np.array(Y)


In [85]:
lookback = 3

In [101]:
X_train, y_train = get_look_back_data(train_data, lookback)
X_test, y_test = get_look_back_data(test_data, lookback)

In [77]:
# 目前数据是[n_samples, n_time_step]的形式，我们把数据整理成[n_samples, n_time_step, n_features]的形式
X_train_nw = np.reshape(X_train, (X_train.shape[0], 3, 1))
X_test_nw = np.reshape(X_test, (X_test.shape[0], 3, 1))

# 训练模型

添加一层`LSTM`，这一层里面有一个unit，输入的数据`input_shape=(lookback, 1)`，后面加入一个紧密层作为输出

In [84]:
?LSTM

In [89]:
X_train.shape

(5597, 3)

In [93]:
# 构建模型整体结构
model = Sequential()
model.add(LSTM(units=1, input_shape=(lookback,1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train_nw, y_train,epochs=20, batch_size=1, verbose=2)

Epoch 1/20
 - 10s - loss: 0.0020
Epoch 2/20
 - 10s - loss: 1.2177e-04
Epoch 3/20
 - 10s - loss: 1.0873e-04
Epoch 4/20
 - 10s - loss: 1.0836e-04
Epoch 5/20
 - 10s - loss: 1.0323e-04
Epoch 6/20
 - 10s - loss: 1.0405e-04
Epoch 7/20
 - 10s - loss: 9.9239e-05
Epoch 8/20
 - 10s - loss: 9.8777e-05
Epoch 9/20
 - 10s - loss: 9.6002e-05
Epoch 10/20
 - 10s - loss: 9.5986e-05
Epoch 11/20
 - 10s - loss: 9.1002e-05
Epoch 12/20
 - 11s - loss: 9.0784e-05
Epoch 13/20
 - 12s - loss: 8.7833e-05
Epoch 14/20
 - 10s - loss: 8.5118e-05
Epoch 15/20
 - 10s - loss: 8.3535e-05
Epoch 16/20
 - 11s - loss: 8.4331e-05
Epoch 17/20
 - 10s - loss: 8.0457e-05
Epoch 18/20
 - 11s - loss: 7.9710e-05
Epoch 19/20
 - 10s - loss: 7.8332e-05
Epoch 20/20
 - 9s - loss: 7.6108e-05


In [96]:
import math

In [106]:
y_train = y_train.reshape(y_train.shape[0],1)
y_test = y_test.reshape(y_test.shape[0],1)

In [110]:
# make predictions
trainPredict = model.predict(X_train_nw)
testPredict = model.predict(X_test_nw)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
y_train = scaler.inverse_transform(y_train)
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform(y_test)
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train[:, 0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[:, 0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 1.28 RMSE
Test Score: 81.07 RMSE


In [114]:
plt.figure(figsize=(18, 6))
# shift train predictions for plotting
trainPredictPlot = np.empty_like(scale_price)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[lookback:len(trainPredict)+lookback, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(scale_price)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(lookback*2)+1:len(scale_price)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(scale_price))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

<Figure size 1296x432 with 0 Axes>

# 学习总结

## 编程技术
- `zip`的解压方法
- 对最终绘制结果图和预测图的代码学习

## 原理
- `lstm`接受的数据结构是`(n_samples, n_time_step, n_features)`这样的数据结构，第一个是样本数量，相当于是记录行数；第二个是时间步，也就是用多大的时间窗口来预测未来，第三个就是每个时间节点可以观测到的变量维度
- 在`lstm`当中`input_shape`为`(3, 2)`分别代表着`(n_time_step, n_features)`前者是代表观测窗口有多大，有多少个值；后者代表有多少个特征

## 回顾一些基本概念
- `batch_size`是每次喂入模型的样本量，1个`epoch`为遍历完所有样本的一个round

## 编程心得
- 要在表头就列清楚一些基本信息，就算不看代码也可以得到**背景知识**，**采取的过程**，**参数**和对应的**结论**

# 进一步学习
- 查看LSTM的源代码

# 参考资料
- [Brent Oil Prices | Kaggle](https://www.kaggle.com/mabusalah/brent-oil-prices/kernels)
- [Time Series Prediction with LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/)
- [How to Reshape Input Data for Long Short-Term Memory Networks in Keras](https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/)